# Models with External Subsystems

In level 2, we have given simple examples of defining external subsystems in `phase_info`. The subsystems that we gave were all dummy subsystems and are not really used in simulation. Assume you have an external subsystem and you want to use it. Let us show you how to achieve this goal.

## Installation of Aviary examples

The Aviary team has provided a few external subsystems for you to use.
These are included in the `aviary/examples/external_subsystems` directory.
We'll now discuss them here and show you how to use them.

## Adding simple_weight subsystems

Currently, there are a couple of examples: `battery` and `simple_weight`. Let us take a look at `simple_weight` first. As shown in this example, this is a simplified example of a component that computes a weight for the wing and horizontal tail. It does not provide realistic computations but rough estimates to `Aircraft.Wing.MASS` and `Aircraft.HorizontalTail.MASS`. When this external subsystem is added to your pre-mission phase, Aviary will compute these weights in its core subsystem as usual, but then the wing mass and tail mass values will be overridden by this external subsytem.

In [level 2](onboarding_level2), we have briefly covered how to add external subsystems in `phase_info`. Alternatively, external subsystems (and any other new keys) can be added after a `phase_info` is loaded. Let us see how it works using the aircraft_for_bench_FwFm.csv model. First, we import this particular external subsystem.

Then add this external subsystem to `pre_mission`.
That is all you need to do in addition to our traditional level 2 examples. Here is the complete run script,

In [ ]:
from copy import deepcopy
from aviary.api import Aircraft
import aviary.api as av

from aviary.examples.external_subsystems.simple_weight.simple_weight_builder import WingWeightBuilder

# Max iterations set to 1 to reduce runtime of example
max_iter = 1
phase_info = deepcopy(av.default_height_energy_phase_info)
# Here we just add the simple weight system to only the pre-mission
phase_info['pre_mission']['external_subsystems'] = [WingWeightBuilder(name="wing_external")]

prob = av.AviaryProblem()

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs('models/test_aircraft/aircraft_for_bench_FwFm.csv', phase_info)

# Have checks for clashing user inputs
# Raise warnings or errors depending on how clashing the issues are
prob.check_and_preprocess_inputs()

prob.add_pre_mission_systems()

prob.add_phases()

prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()

prob.add_driver("SLSQP", max_iter)

prob.add_design_variables()

prob.add_objective()

prob.setup()

prob.set_initial_guesses()

prob.run_aviary_problem(suppress_solver_print=True)

print('Engine Mass', prob.get_val(av.Aircraft.Engine.MASS))
print('Wing Mass', prob.get_val(av.Aircraft.Wing.MASS))
print('Horizontal Tail Mass', prob.get_val(av.Aircraft.HorizontalTail.MASS))

Ignore the intermediate warning messages and you see the outputs at the end.
Since this is a `FLOPS` mission and no objective is provided, we know that the objective is `fuel_burned`.

To see the outputs without external subsystem add-on, let us comment out the lines that add the wing weight builder and run the modified script:

In [ ]:
# # Here we just add the simple weight system to only the pre-mission
# phase_info['pre_mission']['external_subsystems'] = [WingWeightBuilder(name="wing_external")]

# Max iterations set to 1 to reduce runtime of example
max_iter = 1
prob = av.AviaryProblem()

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs('models/test_aircraft/aircraft_for_bench_FwFm.csv', av.default_height_energy_phase_info)

# Have checks for clashing user inputs
# Raise warnings or errors depending on how clashing the issues are
prob.check_and_preprocess_inputs()

prob.add_pre_mission_systems()

prob.add_phases()

prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()

prob.add_driver("SLSQP", max_iter)

prob.add_design_variables()

prob.add_objective()

prob.setup()

prob.set_initial_guesses()

prob.run_aviary_problem(suppress_solver_print=True)

print('Engine Mass', prob.get_val(Aircraft.Engine.MASS))
print('Wing Mass', prob.get_val(Aircraft.Wing.MASS))
print('Horizontal Tail Mass', prob.get_val(Aircraft.HorizontalTail.MASS))

As we see, the engine mass is not altered but wing mass and tail mass are changed dramatically. This is not surprising because our `simple_weight` subsystem is quite simple. Later on, we will show you a more realistic wing weight external subsystem.

## Adding battery subsystem

In the above example, there is no new Aviary variable added to Aviary and the external subsystem is added to pre-mission only. So, the subsystem is not very involved. We will see a more complicated example now. Before we move on, let us recall the steps in Aviary model building:

- **init**
- **load_inputs**
- **check_and_preprocess_inputs**
- **add_pre_mission_systems**
- **add_phases**
- **add_post_mission_systems**
- **link_phases**
- add_driver
- **add_design_variables**
- **add_objective**
- setup
- **set_initial_guesses**
- run_aviary_problem

The steps in bold are related specifically to subsystems. So, almost all of the steps involve subsystems. As long as your external subsystem is built based on the guidelines, Aviary will take care of your subsystem.

The next example is the [battery subsystem](https://github.com/OpenMDAO/Aviary/blob/main/aviary/docs/user_guide/battery_subsystem_example.md). The battery subsystem provides methods to define the battery subsystem's states, design variables, fixed values, initial guesses, and mass names. It also provides methods to build OpenMDAO systems for the pre-mission and mission computations of the subsystem, to get the constraints for the subsystem, and to preprocess the inputs for the subsystem. This subsystem has its own set of variables. We will build an Aviary model with full phases (namely, `climb`, `cruise` and `descent`) and maximize the final total mass: `Dynamic.Mission.MASS`.

We also need `BatteryBuilder` along with battery related aircraft variables and build a new battery object.
Now, add our new battery subsystem into each phase including pre-mission:


In [ ]:
from aviary.examples.external_subsystems.battery.battery_builder import BatteryBuilder
from aviary.examples.external_subsystems.battery.battery_variables import Aircraft

battery_builder = BatteryBuilder(include_constraints=False)

phase_info['pre_mission']['external_subsystems'] = [battery_builder]
phase_info['climb']['external_subsystems'] = [battery_builder]
phase_info['cruise']['external_subsystems'] = [battery_builder]
phase_info['descent']['external_subsystems'] = [battery_builder]

Start an Aviary problem and load in an aircraft input deck:

In [ ]:
prob = av.AviaryProblem()

prob.load_inputs('models/test_aircraft/aircraft_for_bench_FwFm.csv', phase_info)


In the battery subsystem, the type of battery cell we use is `18650`. The option is not set in `input_file`, instead is is controlled by importing the correct battery map [here](https://github.com/OpenMDAO/Aviary/blob/1fca1c03cb2e1d6387442162e8d7dabf83eee197/aviary/examples/external_subsystems/battery/model/reg_thevenin_interp_group.py#L5).

We can then check our inputs:

In [ ]:
# Have checks for clashing user inputs
# Raise warnings or errors depending on severity of the issues
prob.check_and_preprocess_inputs()

### Checking in the setup function call

Function `setup` can have an argument `check` with default value `False`. If we set it to `True`, it will cause a default set of checks to be run. So, instead of a simple `prob.setup()` call, let us do the following:

The following are a few check points printed on the command line:

```
INFO: checking system
INFO: checking solvers
INFO: checking dup_inputs
INFO: checking missing_recorders
```



In [ ]:
prob.add_pre_mission_systems()

traj = prob.add_phases()

prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()

max_iter = 1
prob.add_driver("SLSQP", max_iter)

prob.add_design_variables()

prob.add_objective('mass')

prob.setup(check=True)

prob.set_initial_guesses()

prob.run_aviary_problem()

# user defined outputs
print('Battery MASS', prob.get_val(Aircraft.Battery.MASS))
print('Cell Max', prob.get_val(Aircraft.Battery.Cell.MASS))
masses_descent = prob.get_val('traj.descent.timeseries.mass', units='kg')
print(f"Final Descent Mass: {masses_descent[-1]}")

print('done')

## More on outputs

We are done with our model. For our current example, let us add a few more lines after the aviary run:

In [ ]:
print('Battery MASS', prob.get_val(Aircraft.Battery.MASS))
print('Cell Max', prob.get_val(Aircraft.Battery.Cell.MASS))

Since our objective is `mass`, we want to print the value of `Dynamic.Mission.Mass`. Remember, we have imported Dynamic from aviary.variable_info.variables for this purpose.

So, we have to print the final mass in a different way. Keep in mind that we have three phases in the mission and that final mass is our objective. So, we can get the final mass of the descent phase instead. Let us try this approach. Let us comment out the print statement of final mass (and the import of Dynamic), then add the following lines:

In [ ]:
masses_descent = prob.get_val('traj.descent.timeseries.mass', units='kg')
print(f"Final Descent Mass: {masses_descent[-1]}")

## More on objectives

Now, let us change our objective to battery state of charge after the climb phase. So, comment out `prob.add_objective('mass')` and add the following line right after:

```
prob.model.add_objective(
    f'traj.climb.states:{Mission.Battery.STATE_OF_CHARGE}', index=-1, ref=-1)
```

In the above, `index=-1` means the end of climb phase and `ref=-1` means that we want to maximize the state of charge at the end of climb phase. Once again, we are unable to print battery state of charge as we did with battery mass and battery cell mass. We will use the same approach to get mass. In fact, we have prepared for this purpose by setting up time series of climb and cruise phases as well. All we need to do is to add the following lines:

```
soc_cruise = prob.get_val(
    'traj.climb.timeseries.mission:battery:state_of_charge')
print(f"State of Charge: {soc_cruise[-1]}")
```

Now you get a new output:

```
State of Charge: [0.97458496]

## The check_partials function

In order to make sure that your model computes all the derivatives correctly, OpenMDAO provides a method called `check_partials` which checks partial derivatives comprehensively for all Components in your model. Let us continue in our model:

In [ ]:
prob.check_partials(compact_print=True)

## Adding an OpenAeroStruct wingbox external subsystem

[OpenAeroStruct](https://github.com/mdolab/OpenAerostruct/) (OAS) is a lightweight tool that performs aerostructural optimization using OpenMDAO. This is an example that shows you how to use an existing external package with Aviary.

### Installation of OpenAeroStruct

We would like to have easy access to the examples and source code. So we install OpenAeroStruct by cloning the OpenAeroStruct repository. We show you how to do the installation on Linux. Assume you want to install it at `~/$USER/workspace`. Do the following:

```
cd ~/$USER/workspace
git clone https://github.com/mdolab/OpenAeroStruct.git
~/$USER/workspace/OpenAeroStruct
pip install -e .
```

If everything runs smoothly, you should see something like:

```
Successfully installed openaerostruct
```

Most of the packages that OpenAeroStruct depends on are installed already (see [here](https://mdolab-openaerostruct.readthedocs-hosted.com/en/latest/installation.html)). For our example, we need [ambiance](https://pypi.org/project/ambiance/) and an optional package: [OpenVSP](http://openvsp.org/).

To install `ambiance`, do the following:

```
pip install ambiance
```

You should see something like:

```
Installing collected packages: ambiance
Successfully installed ambiance-1.3.1
```

```{note}
You must ensure that the Python version in your environment matches the Python used to compile OpenVSP. You must [install OpenVSP](https://openvsp.org/wiki/doku.php?id=install) on your Linux box yourself.
```

To check your installation of OpenVSP is successful, please run

```
(av1)$ python openaerostruct/tests/test_vsp_aero_analysis.py
```

Windows users should visit [OpenVSP](http://openvsp.org/download.php) and follow the instruction there.

### Understanding the OpenAeroStruct Example

The OpenAeroStruct example is explained in detail in [Using Aviary and OpenAeroStruct Together](../examples/OAS_subsystem).

### Running the OpenAeroStruct Example

We are ready to run this example. First, we create an `OASWingWeightBuilder` instance.

In [ ]:
import numpy as np
import openmdao.api as om
import aviary.api as av
from aviary.examples.external_subsystems.OAS_weight.OAS_wing_weight_builder import OASWingWeightBuilder

wing_weight_builder = OASWingWeightBuilder()

Let's add a few phases in the mission. In particular, let's add the object we just created as an external subsystem to `pre_mission`.
We are only adding to `pre_mission` here as the OpenAeroStruct design is only done in the sizing portion of pre-mission and doesn't need to be called during the mission.

In [ ]:
# Load the phase_info and other common setup tasks
phase_info = {
    'climb_1': {
        'subsystem_options': {'core_aerodynamics': {'method': 'computed'}},
        'user_options': {
            'optimize_mach': False,
            'optimize_altitude': False,
            'polynomial_control_order': 1,
            'num_segments': 5,
            'order': 3,
            'solve_for_distance': False,
            'initial_mach': (0.2, 'unitless'),
            'final_mach': (0.72, 'unitless'),
            'mach_bounds': ((0.18, 0.74), 'unitless'),
            'initial_altitude': (0.0, 'ft'),
            'final_altitude': (32000.0, 'ft'),
            'altitude_bounds': ((0.0, 34000.0), 'ft'),
            'throttle_enforcement': 'path_constraint',
            'fix_initial': True,
            'constrain_final': False,
            'fix_duration': False,
            'initial_bounds': ((0.0, 0.0), 'min'),
            'duration_bounds': ((64.0, 192.0), 'min'),
        },
        'initial_guesses': {'times': ([0, 128], 'min')},
    },
    'climb_2': {
        'subsystem_options': {'core_aerodynamics': {'method': 'computed'}},
        'user_options': {
            'optimize_mach': False,
            'optimize_altitude': False,
            'polynomial_control_order': 1,
            'num_segments': 5,
            'order': 3,
            'solve_for_distance': False,
            'initial_mach': (0.72, 'unitless'),
            'final_mach': (0.72, 'unitless'),
            'mach_bounds': ((0.7, 0.74), 'unitless'),
            'initial_altitude': (32000.0, 'ft'),
            'final_altitude': (34000.0, 'ft'),
            'altitude_bounds': ((23000.0, 38000.0), 'ft'),
            'throttle_enforcement': 'boundary_constraint',
            'fix_initial': False,
            'constrain_final': False,
            'fix_duration': False,
            'initial_bounds': ((64.0, 192.0), 'min'),
            'duration_bounds': ((56.5, 169.5), 'min'),
        },
        'initial_guesses': {'times': ([128, 113], 'min')},
    },
    'descent_1': {
        'subsystem_options': {'core_aerodynamics': {'method': 'computed'}},
        'user_options': {
            'optimize_mach': False,
            'optimize_altitude': False,
            'polynomial_control_order': 1,
            'num_segments': 5,
            'order': 3,
            'solve_for_distance': False,
            'initial_mach': (0.72, 'unitless'),
            'final_mach': (0.36, 'unitless'),
            'mach_bounds': ((0.34, 0.74), 'unitless'),
            'initial_altitude': (34000.0, 'ft'),
            'final_altitude': (500.0, 'ft'),
            'altitude_bounds': ((0.0, 38000.0), 'ft'),
            'throttle_enforcement': 'path_constraint',
            'fix_initial': False,
            'constrain_final': True,
            'fix_duration': False,
            'initial_bounds': ((120.5, 361.5), 'min'),
            'duration_bounds': ((29.0, 87.0), 'min'),
        },
        'initial_guesses': {'times': ([241, 58], 'min')},
    },
    'post_mission': {
        'include_landing': False,
        'constrain_range': True,
        'target_range': (1800., 'nmi'),
    },
}

phase_info['pre_mission'] = {'include_takeoff': False, 'optimize_mass': True}
phase_info['pre_mission']['external_subsystems'] = [wing_weight_builder]

We can now create an Aviary problem, load in an aircraft input deck, and do routine input checks:

In [ ]:
aircraft_definition_file = 'models/test_aircraft/aircraft_for_bench_FwFm.csv'
make_plots = False
max_iter = 0
optimizer = 'SNOPT'

prob = av.AviaryProblem()

prob.load_inputs(aircraft_definition_file, phase_info)
prob.check_and_preprocess_inputs()
prob.add_pre_mission_systems()
prob.add_phases()
prob.add_post_mission_systems()
prob.link_phases()

Next we select the driver and call setup on the problem:

In [ ]:
driver = prob.driver = om.pyOptSparseDriver()
driver.options["optimizer"] = optimizer
driver.declare_coloring()
driver.opt_settings["Major iterations limit"] = max_iter
driver.opt_settings["Major optimality tolerance"] = 1e-4
driver.opt_settings["Major feasibility tolerance"] = 1e-5
driver.opt_settings["iSumm"] = 6

prob.add_design_variables()
prob.add_objective()
prob.setup()
prob.set_initial_guesses()

Now we need to set some OpenAeroStruct-specific parameters before running Aviary:

In [ ]:
OAS_sys = 'pre_mission.wing_weight.aerostructures.'
prob.set_val(OAS_sys + 'box_upper_x', np.array([0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6]), units='unitless')
prob.set_val(OAS_sys + 'box_lower_x', np.array([0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6]), units='unitless')
prob.set_val(OAS_sys + 'box_upper_y', np.array([ 0.0447,  0.046,  0.0472,  0.0484,  0.0495,  0.0505,  0.0514,  0.0523,  0.0531,  0.0538, 0.0545,  0.0551,  0.0557, 0.0563,  0.0568, 0.0573,  0.0577,  0.0581,  0.0585,  0.0588,  0.0591,  0.0593,  0.0595,  0.0597,  0.0599,  0.06,    0.0601,  0.0602,  0.0602,  0.0602,  0.0602,  0.0602,  0.0601,  0.06,    0.0599,  0.0598,  0.0596,  0.0594,  0.0592,  0.0589,  0.0586,  0.0583,  0.058,   0.0576,  0.0572,  0.0568,  0.0563,  0.0558,  0.0553,  0.0547,  0.0541]), units='unitless')
prob.set_val(OAS_sys + 'box_lower_y', np.array([-0.0447, -0.046, -0.0473, -0.0485, -0.0496, -0.0506, -0.0515, -0.0524, -0.0532, -0.054, -0.0547, -0.0554, -0.056, -0.0565, -0.057, -0.0575, -0.0579, -0.0583, -0.0586, -0.0589, -0.0592, -0.0594, -0.0595, -0.0596, -0.0597, -0.0598, -0.0598, -0.0598, -0.0598, -0.0597, -0.0596, -0.0594, -0.0592, -0.0589, -0.0586, -0.0582, -0.0578, -0.0573, -0.0567, -0.0561, -0.0554, -0.0546, -0.0538, -0.0529, -0.0519, -0.0509, -0.0497, -0.0485, -0.0472, -0.0458, -0.0444]), units='unitless')
prob.set_val(OAS_sys + 'twist_cp', np.array([-6., -6., -4., 0.]), units='deg')
prob.set_val(OAS_sys + 'spar_thickness_cp', np.array([0.004, 0.005, 0.008, 0.01]), units='m')
prob.set_val(OAS_sys + 'skin_thickness_cp', np.array([0.005, 0.01, 0.015, 0.025]), units='m')
prob.set_val(OAS_sys + 't_over_c_cp', np.array([0.08, 0.08, 0.10, 0.08]), units='unitless')
prob.set_val(OAS_sys + 'airfoil_t_over_c', 0.12, units='unitless')
prob.set_val(OAS_sys + 'fuel', 40044.0, units='lbm')
prob.set_val(OAS_sys + 'fuel_reserve', 3000.0, units='lbm')
prob.set_val(OAS_sys + 'CD0', 0.0078, units='unitless')
prob.set_val(OAS_sys + 'cruise_Mach', 0.785, units='unitless')
prob.set_val(OAS_sys + 'cruise_altitude', 11303.682962301647, units='m')
prob.set_val(OAS_sys + 'cruise_range', 3500, units='nmi')
prob.set_val(OAS_sys + 'cruise_SFC', 0.53 / 3600, units='1/s')
prob.set_val(OAS_sys + 'engine_mass', 7400, units='lbm')
prob.set_val(OAS_sys + 'engine_location', np.array([25, -10.0, 0.0]), units='m')

We are now ready to run Aviary on this model.
Note that there are multiple numbers of optimization loops that are output from this run even though we have set `max_iter` to 0.
This is because OpenAeroStruct has an optimization process internally. In order to shorten the runtime, we have set `run_driver = False`. This means that we will not run optimization but [run model](https://openmdao.org/newdocs/versions/latest/features/core_features/running_your_models/run_model.html). 

Finally, we print the newly computed wing mass:

In [ ]:
print('wing mass = ',prob.model.get_val(av.Aircraft.Wing.MASS, units='lbm'))

The result is comparable to the output without OpenAeroStruct external subsystem.